# DCOPF

In [1]:
import numpy as np

import andes
import ams

In [2]:
andes.config_logger(stream_level=10)

In [3]:
sa = andes.load(andes.get_case('ieee14/ieee14_ace.xlsx'),
              setup=True,)

Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
Found files: ['/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/ieee14/ieee14_ace.xlsx']
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Input format guessed as xlsx.
Parsing input file "/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/ieee14/ieee14_ace.xlsx"...
Input file parsed in 0.3248 seconds.
Setting internal address for Bus
Setting internal address for PQ
Setting internal address for PV
Setting internal address for Slack
Setting internal address for Shunt
Setting internal address for Line
Setting internal address for Area
System internal structure set up in 0.0249 seconds.


In [4]:
ams.config_logger(stream_level=10)

In [5]:
sp = ams.load(ams.get_case('matpower/case14.m'),
              setup=True,)

ppc = ams.io.pypower.system2ppc(sp)

Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
Found files: ['/Users/jinningwang/Documents/work/ams/ams/cases/matpower/case14.m']
Input format guessed as matpower.
Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/matpower/case14.m"...
Detected raw file encoding: {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
CASE14    Power flow data for IEEE 14 bus test case.
Input file parsed in 0.0146 seconds.
System set up in 0.0035 seconds.


In [6]:
sp.mats.PTDF.shape

(20, 14)

In [7]:
sp.StaticGen.get(src='bus', attr='v', idx=sp.StaticGen.get_idx())

array([1., 2., 3., 6., 8.])

In [8]:
sp.Bus.idx.v

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [9]:
m1, m2 = sp.mats.rePTDF()

In [10]:
m1.shape

(20, 5)

In [11]:
m2.shape

(20, 9)